# **Tutorial: Generating Randomized TeamCraft Configurations**

This Jupyter notebook demonstrates how to generate coresponding JSON configuration files for your task. Each file defines:
- The number of agents (bots), their starting positions, and their inventory.
- The playgrounds (locations, sizes, and materials).
- Blocks to place (coordinates, shapes, types).
- Commands to execute within the environment (e.g., clearing areas, spawning mobs, and much [more](https://minecraft.fandom.com/wiki/Commands)!).

---

## 0. Requirement
- Python 3.9+
- numpy


## 1. Helper Functions

In [7]:
from helper import *
import json
import random

## 2. Design your own task

This tutorial will guide you thourgh how TeamCraft configure *building* task.

### 2.1 Define Parameters

#### **Agent count**
The current simulation platform supports an agent range from **2 to 4**.

For **agent counts of 2 and 3**, TeamCraft loads **3 working agents** and **1 observer** by default, with the following roles:

- **bot1**: Working Agent 1
- **bot2**: Working Agent 2
- **bot3**: Working Agent 3
- **bot4**: Observer (default game mode: spectator, not visible in other agents' RGB rendering)

**Note:** If the agent count is **2**, `bot3` will still join the game but will be teleported to a reserved point at **(10, 10, 10)**—far from the regular ground surface. By default, the environment will **not render its first-person perspective image**, but you can change this behavior by modifying the environment configuration file.

For **agent count 4**, an additional working agent is introduced:

- **bot0**: Working Agent 4

Each agent has different skin, as shown below:

# ![skin](skin.png)

In [8]:
agent_counts = [2, 3, 4]

#### **Background**
The `background` variable defines the type of ground block used in the simulation. Below, you have been provided with a set of predefined blocks. However, you can add **over 800 more blocks** by using their [official Minecraft names](https://minecraft.wiki/w/Block).

##### **Finding the Right Block Name**
To use a new block type, follow these steps:

1. **Locate your preferred block** under the "List of Blocks" section. Squared blocks are recommended.
2. **Click on the block link** to open its detailed page.
3. **Check its availability in version 1.16.4 (Java Edition):**
   - Navigate to the **"History"** section.
   - Find the **"Java Edition"** tab.
   - Look for the version number in the first row under Java Edition.
   - Ensure the block was introduced **before or in** version **1.16.4**.
     - Example: [Bricks](https://minecraft.wiki/w/Bricks) were introduced in version **1.0.0**, so they are available.
     - Example: [Froglight](https://minecraft.wiki/w/Froglight#Pearlescent) was introduced in **1.19**, which is later than **1.16.4**, so it is **not available**.
4. **Retrieve the block name:**
   - Go to the **"Data values"** section.
   - Find the **"ID"** subsection.
   - Locate the **"Java Edition"** identifier.
   - Use the block name listed under **"Identifier"** in the script.
5. **Add 'minecraft:' before the block name**
   - A valid name should look like **minecraft:BLOCK_NAME**, where BLOCK_NAME is the name your just found.

**Note:** Some blocks have different **variants**, primarily in color. Each variant is a valid block and can be used independently.

In [9]:
background = ['minecraft:cyan_concrete', 'minecraft:stone', 'minecraft:oak_wood', 'minecraft:hay_block', 'minecraft:glass',
                'minecraft:glowstone', 'minecraft:gold_block', 'minecraft:pink_wool', 'minecraft:obsidian','minecraft:smooth_quartz']

#### **Placeable Item**

In this tutorial, we use *building* task as example, hence, all item are need to be a **Block** that are placeable, since this task requires agent to place block based on the blue print given. Therefore, in this case, the items agents interacting with will be the block, same as above.

**Note:** You do not need to add "minecraft:" prefix in this case as it is handled later in the code

In [10]:
all_items = ['oak_fence', 'birch_log', 'bookshelf', 'acacia_fence', 'oak_log', 'coal_ore',
                'bricks', 'sandstone', 'stone', 'iron_ore', 'gold_ore', 'sponge', 'sea_lantern',
                'dirt', 'grass_block', 'clay', 'oak_planks', 'emerald_block', 'bricks', 'pumpkin',
                'orange_concrete', 'purple_wool', 'end_stone'
            ]

#### **Playground Location**

Each playground environment has a **central location** and a **default size** of 5. We provide the following available locations:

- **Village** → Center: `[223, 70, 128]`
- **Desert Village** → Center: `[4261, 74, 161]`
- **Swamp** → Center: `[4284, 64, 1126]`
- **Ice on Water** → Center: `[7393, 63, 5325]`
- **Snow Mountain** → Center: `[11671, 106, 8488]`
- **Mountain Half** → Center: `[4302, 97, 211]`
- **Forest** → Center: `[14140, 67, 10171]`

You can also find your own playground location by:
   - Launching a local Minecraft instance (refer to [env doc](../env_doc.md) under *Visualization* for setup instructions).
   - Loading `/teamcraft/teamcraft/tasks/tasks_world/world_xxx` as a game save.
   - Enabling cheats in the save:
     - Press `esc`.
     - Choose `Open to LAN`.
     - Select `Allow Cheat`.
     - Return to the game.
   - Exploring the world using creative mode (`/gamemode creative`). Cheats must be enabled for this command to work.
   - Once you find an ideal surface, press `F3`, and record the position of the block that will be the center of your playground floor.
     - Look for **"Targeted Block"** on the right side of your debug screen.

**Note:** If your playgound size is bigger than 5, you need to change the `obs command` later in the code, to let observer captcture the entire playgound.
    

In [11]:
# name, center, size
# Recommended size is 5 or 6
playground_data = [
                    ("villege", [223, 70, 128], 5),
                    ("desert_villege", [4261, 74, 161], 5),
                    ("swamp", [4284, 64, 1126], 5),
                    ("ice_on_water", [7393, 63, 5325], 5),
                    ("snow_mountain", [11671, 106, 8488], 5),
                    ("mountain_half", [4302, 97, 211], 5),
                    ("forest", [14140, 67, 10171], 5)
                  ]

#### **Base Shape Configuration**


In our case, we do not used the full space of the playground to place the blocks. Instead, we limit our blocks in the following shape.

You can define your shape, as long as it satisfied [x,y,z]:
   - x <= playground size (5 in our example)
   - y <= 2
   - z <= playgound size (5 in our example)

In [12]:
# Shape of the base used for block placement
# Width, Length, Height
base_shape = [[4,1,2],[3,1,2], [2,2,2],[2,3,2]]

#### **Output Dir**

The output directory is defined for storing configuration files.

In [ ]:
out_dir = '/YOUR_PATH_TO_TASK_FOLDER/configure/'
initialize_output_directory(out_dir)

#### **Variance Config**

Let pick a combination! Randomly!

In [28]:
def _var_cfg(random_seed):
    random.seed(num_action)
    var_cfg = {
        "agent_counts": random.sample(agent_counts, 1),
        "background": random.sample(background, 1),
        "playground_location": sample_dict_item(generate_multiple_playgrounds(playground_data)),
        "base_shape": random.sample(base_shape,1),
        "block_type": None, 
        "placement_shape": None}
    return var_cfg

    

#### **Bot Name**

In [29]:
def _bot(var_cfg):
    if var_cfg["agent_counts"][0] == 2:
        return ['bot1', 'bot2']
    elif var_cfg["agent_counts"][0] == 3:
        return ['bot1', 'bot2', 'bot3']
    elif var_cfg["agent_counts"][0] == 4:
        return ['bot1', 'bot2', 'bot3', 'bot0']

#### **Bot Init Orientation**

Agents will be spawn near the edge of the playgound. You can check this behavior by editng the `generate_multiple_playgrounds` function in the `helper.py`

All agents can look at a completely random direction.

In [18]:
def bot_init(cfg):
    bot_range_x = [var_cfg["playground_location"]["bot_range"][0] for _ in range(len(bot))]
    bot_range_y = [var_cfg["playground_location"]["bot_range"][1] for _ in range(len(bot))]
    bot_height = var_cfg["playground_location"]["bot_range"][2]

    bot_pitch = [[-10,10],[-10,10],[-10,10],[-10,10]]
    bot_yaw = [[0,360],[0,360],[0,360],[0,360]]
    
    return bot_range_x, bot_range_y, bot_height, bot_pitch, bot_yaw

#### **Init Command**

The init command that will be execute beofore each task. This gives **huge** freedom to define the environment. 

You can refer to command usage [here](https://minecraft.fandom.com/wiki/Commands).

In [19]:
init_command = f"""
                await bot1.chat('hello!!!!!!!!!!!');
                await bot1.chat('/gamerule doMobSpawning false');
                await bot1.chat('/gamerule randomTickSpeed 0');
                """

#### **Playground Configuration**

Playground configuration is also done by using pre-defined command string. 

The process include:
  - force load area
  - clean the playgound by filling air block (remove block)
  - move observer to the top of the playgound and look down
  - fill ground layer with pre-defined material

**Note:** If you have playground size bigger than 5, you should change the code accordingly. The current setup is used for size of 5.

In [26]:
def playground_command(cfg):
    # ------ Playground Configuration ------
    # obs_command should be center and +4 block above the center with playgound size of 5, 
    # IMPORTANT: for a larger playground size, change obs_command and observer setup command accordingly
    # obs_command yaw pitch should be -90 90  
    
    xx, yy, zz = var_cfg['playground_location']['x'][0], var_cfg['playground_location']['y'], var_cfg['playground_location']['z'][0]
    xxx, zzz =   var_cfg['playground_location']['x'][1], var_cfg['playground_location']['z'][1]
    obs_x, obs_y, obs_z = var_cfg['playground_location']['center']
    obs_y += 4
    
    # Forceload the area
    playground_command = f"await bot1.chat('/forceload add {xx-1} {zzz+1}');"
    # Clear the area
    playground_command+= f"await bot1.chat('/fill {xx-4} {yy-1} {zz-4} {xxx+4} {yy+1} {zzz+4} minecraft:air');"
    # Set up the observer
    playground_command+=f"""
                    await bot4.chat('/setblock {obs_x} {obs_y-1} {obs_z} minecraft:barrier');
                    await bot4.chat('/gamemode spectator');
                    await bot4.chat('/tp @p {obs_x} {obs_y} {obs_z} -90 90');
                """

    # Fill the ground layer with the background material
    playground_command += f"await bot1.chat('/fill {xx} {yy-1} {zz} {xxx} {yy-1} {zzz} {var_cfg['background'][0]}');"

    # Fill the layers above with air
    playground_command += f"await bot1.chat('/fill {xx-1} {yy} {zz-1} {xxx} {yy+1} {zzz} minecraft:air');"
    
    return playground_command


#### **Observer Configuration**

**Note:** If you have playground size bigger than 6, you should change the code accordingly. The current setup is good for playground size <= 6.

In [21]:
def obs_command(cfg):
    
    # ------ Playground Configuration ------
    # obs_command should be center and +4 block above the center with playgound size of 5, 
    # IMPORTANT: for a larger playground size, change obs_command and observer setup command accordingly
    # obs_command yaw pitch should be -90 90  
    
    obs_x, obs_y, obs_z = var_cfg['playground_location']['center']
    obs_y += 4
    
    obs_command = []
    obs_command.append([f"""
                        await bot4.chat('/setblock {obs_x-6} {obs_y-5} {obs_z} minecraft:barrier');
                        await bot4.chat('/tp @p {obs_x-6} {obs_y-4} {obs_z} -90 0');
                        """])
    obs_command.append([f"""
                        await bot4.chat('/setblock {obs_x} {obs_y-5} {obs_z+6} minecraft:barrier');
                        await bot4.chat('/tp @p {obs_x} {obs_y-4} {obs_z+6} -180 0');
                        """])
    obs_command.append([f"""
                        await bot4.chat('/tp @p {obs_x} {obs_y} {obs_z} -90 90');
                 """])
    return obs_command

#### **Done Input**

Define your done input here. This is a extra variable that could be used by your done function, and will be automatically load by env.

In [22]:
# TODO: Implement the done input configuration in the code below!!!!
# DO NOT IMPLEMENT THE ACTION COMMANDS HERE
done_input=[]

#### **Action**

Define agent action here. Please refer to [env doc](../env_doc.md) for supprted actions.

In [ ]:
# TODO: Implement the bot actions configuration in the code below!!!!
# DO NOT IMPLEMENT THE ACTION COMMANDS HERE
action_list = [[] for _ in range(len(bot))]

#### **Inventory**

Define agent inventories here. Inventory can be any minecraft items that supported by Java Edition 1.16.4. There are over 1,000 items available.

##### Refer to [Complete list](https://minecraft.fandom.com/wiki/Item).

##### **Finding the Right Item Name**
To use a new block type, follow these steps:

1. **Locate your preferred item** under the "List of items" section.
2. **Click on the block link** to open its detailed page.
3. **Check its availability in version 1.16.4 (Java Edition):**
   - Navigate to the **"History"** section.
   - Find the **"Java Edition"** tab.
   - Look for the version number in the first row under Java Edition.
   - Ensure the item was introduced **before or in** version **1.16.4**.
     - Example: [Bricks](https://minecraft.wiki/w/Bricks) were introduced in version **1.0.0**, so they are available.
     - Example: [Froglight](https://minecraft.wiki/w/Froglight#Pearlescent) was introduced in **1.19**, which is later than **1.16.4**, so it is **not available**.
4. **Retrieve the item name:**
   - Go to the **"Data values"** section.
   - Find the **"ID"** subsection.
   - Locate the **"Java Edition"** identifier.
   - Use the block name listed under **"Identifier"** in the script.

**Note:** Some item have different **variants**, primarily in color. Each variant is a valid block and can be used independently.

In [ ]:
# TODO: Implement the bot inventory configuration in the code below!!!!
# DO NOT IMPLEMENT THE INVENTORY COMMANDS HERE
bot_assigned_item = [[] for i in range(len(bot))]

### 2.3 Main Loop

Construct a full file by 100 times!

In [ ]:
# Batch random generation of JSON files
for num_action in range(0, 100):

    # ----- Random Seed -----
    random.seed(num_action)
    
    # ----- Configuration -----
    var_cfg = _var_cfg(num_action)

    # ----- Bot Configuration -----
    bot = _bot(var_cfg)
    bot_range_x, bot_range_y, bot_height, bot_pitch, bot_yaw = bot_init(var_cfg)

    # ----- Command Configuration -----
    input_data = {}
    
    # Example command, for reference
    input_data['command']=init_command
                
    if len(bot) == 2:
        input_data['command']+="""
                    await bot3.chat('/gamemode spectator');
                    await bot3.chat('/tp @p 10 10 10');
                    """
    if len(bot) == 3 or len(bot) == 4:
        input_data['command']+="""
                    await bot3.chat('/gamemode survival');
                    """
    
    # ------ Playground Configuration ------
    input_data['command']+=playground_command(var_cfg)

    # ----- Observer Configuration -----
    input_data['obs_command'] = obs_command(var_cfg)
    
    # ----- Done Input Configuration -----
    # TODO: Implement the done input configuration
    done_input = []
    # ----- Bot Action and Inventory Configuration -----
    # TODO: Implement the bot actions configuration
    action_list = [[] for _ in range(len(bot))]
    # TODO: Implement the bot inventory configuration
    bot_assigned_item = [[] for i in range(len(bot))]
    
    
    
    
    
    
    ############################################################################################
    # ------------------------ Below are example for building task -----------------------------
    max_capacity = var_cfg["base_shape"][0][0] * var_cfg["base_shape"][0][1] * 2

    # ----- Item Configuration -----
    # Target item selection
    n_items = random.randint(6, max_capacity) if len(bot) == 3 else random.randint(5, max_capacity)
    # Randomly pick k items with replacement
    target_item = random.choices(all_items, k=n_items)
    
    var_cfg["block_type"] = target_item
    var_cfg["block_count"] = n_items
    
    # ----- Block Placement Configuration -----
    width, depth, height = var_cfg["base_shape"][0]
    spots = list(range(0, width * depth * height))
    
    # ----- Example of randomizing placing of "target_item" on the playground ----- 
    
    placed_spots = []
    placed_spots_number = []

    for item in target_item:
        # Randomly select a spot
        spot = spots.pop()
        x, y, z = translate_to_coordinates(spot, width, depth, height)
        
        placed_spots.append((x, z, y))
        placed_spots_number.append(translate_to_index(x, y, z, width, depth, height))
        
        x += var_cfg["playground_location"]["x"][0] + 1
        y += var_cfg["playground_location"]["z"][0] + 1
        z_height = var_cfg['playground_location']["y"] + z
        
        input_data['command'] += f"await bot1.chat('/setblock {x} {z_height} {y} minecraft:{item}');"
        
        # Exapmple done_input. You can design your own done_input based on the reward function
        done_input.append([item,x,z_height,y])
        
        
    var_cfg["placement_shape"] = placed_spots
    var_cfg["placement_shape_number"] = placed_spots_number
    # ------------------------ Above are example for building task -----------------------------
    ############################################################################################

    
    
        
    # ----- Enumearate through the bots and assign actions and inventory -----
    for i, b in enumerate(bot):
        input_data[b]={}
        x_min,x_max = bot_range_x[i]
        y_min,y_max = bot_range_y[i]
        pitch_min,pitch_max = bot_pitch[i]
        yaw_min,yaw_max =  bot_yaw[i]
        x = round(random.uniform(x_min, x_max),2)
        y = round(random.uniform(y_min, y_max),2)
        pitch = round(random.uniform(pitch_min, pitch_max),2)
        yaw = round(random.uniform(yaw_min, yaw_max),2)
        input_data[b]['location'] = [x,bot_height,y]
        input_data[b]['rotation'] = [yaw,pitch]
        input_data[b]['inventory'] = {}
        
        input_data['command']+='await '+b+'.chat(\'/tp @p '+str(x)+' '+str(bot_height)+' '+str(y)+' '+str(yaw)+' ' +str(pitch)+' '+'\');'
        for item in bot_assigned_item[i]:
            input_data[b]['inventory'][item]=input_data[b]['inventory'].get(item,0)+random.randint(1, 3) # Randomly assign 1-3 more items to the bot, change if needed
        ###################################################################################################
        # ----- To make sure bots are able to finish the building task, give them needed target items -----
        for item in random.sample(target_item, random.randint(1, len(target_item))):
            input_data[b]['inventory'][item]=input_data[b]['inventory'].get(item,0)+random.randint(1, 3)
        # -------------------------------------------------------------------------------------------------
        ###################################################################################################
        for item in input_data[b]['inventory']:
            input_data['command']+='await '+b+'.chat(\'/give @p ' +item+' '+str(input_data[b]['inventory'][item])+'\');'

    # ----- Write Configuration -----
    input_data['variant']=num_action
    input_data['done_input']=done_input
    input_data['actions']=interleave_lists(action_list)
    input_data['item_spots']={}
    input_data['variants_config'] = var_cfg
    input_data['bot_list'] = bot
    input_data['center_position'] = var_cfg['playground_location']['center']
    
    # ----- Store JSON file --------
    file = out_dir+str(num_action)+'.json'

    with open(file, 'w') as json_file:
        json.dump(input_data, json_file, indent=4, cls=NpEncoder) 